In [44]:
from selenium import webdriver
from bs4 import BeautifulSoup

driver = webdriver.Chrome() #needs to be in your path, or you can specify here. 
driver.get('http://192.168.66.23') #whatever your ip is


soup = BeautifulSoup(driver.page_source)
driver.close()



In [45]:
cnxTable = soup.find('table', {'id': 'wireless_table'})

rows  = cnxTable.findAll('tr')

macs = []
for row in rows:
    cells = row.findAll('td')
    if cells:
        for cell in cells:
            if 'xx:xx:xx:xx' in cell.text:
                macs.append(cell.text[-5:])
macs

['6F:9D', 'E0:E2', 'F4:D9']

In [31]:
import mysql.connector
from dotenv import load_dotenv
import os

In [46]:
load_dotenv()

sql_id = os.getenv("SQL_ID")
sql_pw = os.getenv("SQL_PW")

cnx = mysql.connector.connect(user=sql_id, password=sql_pw, host='127.0.0.1', database='clientes')
cursor = cnx.cursor()

In [47]:
for l in macs:
    query = ("select datos_per.coduser, datos_per.nom_apell, datos_per.fech_ven, datos_red.dir_mac from datos_red, datos_per "
             "where datos_per.coduser=datos_red.coduser and datos_red.dir_mac like  '%"+l+"%'")

    cursor.execute(query)
    

    for (coduser, nom_apell, fech_ven, dir_mac) in cursor:
        print("{} \t {} \t \t {} \t mac {}".format(coduser, nom_apell, fech_ven, dir_mac, l[2], l[3]))

cursor.close()
cnx.close()


467 	 Renzo Ramirez 	 	 2021-04-13 	 mac 14:5f:94:8b:6f:9d
479 	 Gabriel Espitia 	 	 2021-04-23 	 mac ec:55:f9:54:f4:d9


In [15]:
import requests
import re
r = requests.get('http://192.168.66.5/Status_Wireless.asp', auth=('luis', 'AmE3024OT1esVlj63'))


In [23]:
def elimrep(a):
    b = [a.count(x) for x in a]
    c = [x[0] for x in zip(a, b) if x[1] == 1]
    return c

data = r.text
p = re.compile(r'(?:[0-9a-fA-F]:?){12}')

macs = []
found = re.findall(p, data)
for a in found:
    macs.append(a)

# las repetidas son la mac del router
macs = elimrep(macs)
macs

['50:C8:E5:C7:34:8B', 'A8:C8:3A:7B:E5:B8']